**Задание**
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек 
- 2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
- 2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
 				
- Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
max_words = 200
max_len = 40

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# !pip install openpyxl
# !pip install xlrd
# !pip install nltk
# !pip install keras
# !pip install tensorflow

In [4]:
import pandas as pd
df = pd.read_excel("отзывы за лето.xls")

In [5]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
# Посмотрим на дисбаланс классов
class_counts = df['Rating'].value_counts()
class_counts

Rating
5    14586
1     2276
4     2138
3      911
2      748
Name: count, dtype: int64

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Разделение DataFrame на train и temporary (test + validation) наборы
df_train, df_temp = train_test_split(df, test_size=0.3, random_state=42)

# Разделение temporary (test + validation) набора на test и validation наборы
df_test, df_val = train_test_split(df_temp, test_size=0.5, random_state=42)

In [8]:
# Предобработка
from string import punctuation
from stop_words import get_stop_words
from nltk.stem import SnowballStemmer
import re

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
stemmer = SnowballStemmer("russian")

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [stemmer.stem(word) for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)


In [9]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [10]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /Users/annvorosh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Отфильтруем данные и соберём в корпус N наиболее частых токенов
tokens_filtered = [word for word in tokens if word.isalnum()]

In [12]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [13]:
tokens_filtered_top[:10]

['приложен',
 'удобн',
 'отличн',
 'работа',
 'нрав',
 'телефон',
 'хорош',
 'быстр',
 'супер',
 'обновлен']

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [15]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [16]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [17]:
x_train.shape

(14461, 40)

In [18]:
max_len

40

In [19]:
x_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  3,  1,  2, 29], dtype=int32)

### Keras model

In [20]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam

In [21]:
from keras.utils import to_categorical
num_classes = 5

# Задание частот для каждого класса
# Rating
# 5    14586
# 1     2276
# 4     2138
# 3      911
# 2      748
class_counts = np.array([2276, 748, 911, 2138, 14586])
total_samples = class_counts.sum()

# Рассчитаем веса
count_class_weights = (total_samples / class_counts)

# Преобразуем веса в словарь
class_weights = {i: weight for i, weight in enumerate(count_class_weights)}

# Применяем веса к меткам классов
y_train = to_categorical(df_train["Rating"] - 1, num_classes)
y_val = to_categorical(df_val["Rating"] - 1, num_classes)
class_weights

{0: 9.07688927943761,
 1: 27.61898395721925,
 2: 22.67727771679473,
 3: 9.662768942937324,
 4: 1.4163581516522692}

In [22]:
# Создание модели
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


# # Создание модели
# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
# model.add(Conv1D(128, 3))
# model.add(Activation("relu"))
# model.add(Conv1D(64, 3))  # Добавляем дополнительный сверточный слой
# model.add(Activation("relu"))
# model.add(GlobalMaxPool1D())
# model.add(Dense(128))  # Изменим размер слоя Dense для более высокоуровневых признаков
# model.add(Activation("relu"))
# model.add(Dropout(0.5))  # Добавляем Dropout для регуляризации
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

In [23]:
# Компиляция модели с использованием взвешенной функции потерь
model.compile(
    optimizer=Adam(),
    loss=CategoricalCrossentropy(), 
    metrics=['accuracy'], 
    sample_weight_mode=class_weights
)

In [24]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 1s 25ms/step - loss: 1.4785 - accuracy: 0.1328 - val_loss: 1.2680 - val_accuracy: 0.7049
Epoch 2/20
26/26 [==============================] - 1s 23ms/step - loss: 1.0274 - accuracy: 0.7054 - val_loss: 0.8702 - val_accuracy: 0.7139
Epoch 3/20
26/26 [==============================] - 1s 24ms/step - loss: 0.8101 - accuracy: 0.7486 - val_loss: 0.7941 - val_accuracy: 0.7436
Epoch 4/20
26/26 [==============================] - 1s 23ms/step - loss: 0.7305 - accuracy: 0.7611 - val_loss: 0.7254 - val_accuracy: 0.7505
Epoch 5/20
26/26 [==============================] - 1s 24ms/step - loss: 0.6817 - accuracy: 0.7683 - val_loss: 0.6978 - val_accuracy: 0.7533
Epoch 6/20
26/26 [==============================] - 1s 24ms/step - loss: 0.6659 - accuracy: 0.7721 - val_loss: 0.6909 - val_accuracy: 0.7533
Epoch 7/20
26/26 [==============================] - 1s 24ms/step - loss: 0.6543 - accuracy: 0.7759 - val_loss: 0.6858 - val_accuracy: 0.7540
Epoch 8/20
26

In [25]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 6ms/step - loss: 0.6739 - accuracy: 0.7635


Test score: 0.6739253401756287
Test accuracy: 0.7634720802307129


In [26]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 0s 6ms/step


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score 

In [28]:
vect = TfidfVectorizer(ngram_range=(1, 2), analyzer='word', lowercase=False)

In [29]:
train_ft = vect.fit_transform(df_train['Content'])
valid_ft = vect.transform(df_val['Content'])

In [30]:
lgr = LogisticRegression()

In [31]:
lgr.fit(train_ft, df_train['Rating'].to_numpy())

LogisticRegression()

In [32]:
y_pred = lgr.predict(valid_ft)

In [33]:
accuracy_score(df_val['Rating'].to_numpy(), y_pred)

0.7676669893514037

In [34]:
from gensim.models import Word2Vec

In [35]:
df_train['Content']

10890                                 супер
19623    отличн приложен удобн управля счет
7430                                     ок
7934                                0 минус
9630                         приятн положен
                        ...                
11284                                  нрав
11964               смешн программ пуга рут
5390               мог скача ошибк номер 24
860                                сбербанк
15795                            цел отличн
Name: Content, Length: 14461, dtype: object

In [36]:
# Создание модели Word2Vec с использованием nltk для токенизации
modelW2V = Word2Vec(sentences=df_train['Content'].apply(nltk.word_tokenize), vector_size=100, window=5, min_count=5, workers=8)

In [37]:
# Вывод всех слов и их соответствующих объектов Vocab
modelW2V.wv.key_to_index

{'приложен': 0,
 'удобн': 1,
 'отличн': 2,
 'работа': 3,
 'нрав': 4,
 'телефон': 5,
 'хорош': 6,
 'быстр': 7,
 'супер': 8,
 'обновлен': 9,
 'парол': 10,
 'вход': 11,
 'приход': 12,
 'антивирус': 13,
 'мог': 14,
 'банк': 15,
 'устраива': 16,
 'сбербанк': 17,
 'раз': 18,
 'ввод': 19,
 'перевод': 20,
 'прошивк': 21,
 'карт': 22,
 'рут': 23,
 'нормальн': 24,
 'программ': 25,
 'сдела': 26,
 'разработчик': 27,
 'счет': 28,
 'постоя': 29,
 'ошибк': 30,
 'пользова': 31,
 'норм': 32,
 'понятн': 33,
 'возможн': 34,
 'ок': 35,
 'оплат': 36,
 'пишет': 37,
 'код': 38,
 'платеж': 39,
 'мобильн': 40,
 'смс': 41,
 'последн': 42,
 'функц': 43,
 'шаблон': 44,
 'проблем': 45,
 'сво': 46,
 'вылета': 47,
 'деньг': 48,
 'дела': 49,
 'стал': 50,
 'ин': 51,
 'польз': 52,
 'прав': 53,
 'класс': 54,
 'зайт': 55,
 'проверк': 56,
 'операц': 57,
 'root': 58,
 'meizu': 59,
 '5': 60,
 'доступ': 61,
 'крут': 62,
 'онлайн': 63,
 'работ': 64,
 'заход': 65,
 'мо': 66,
 'верс': 67,
 'установ': 68,
 'плох': 69,
 'классн':

In [38]:
vect_idf = TfidfVectorizer()
vect_idf.fit_transform(df_train['Content'])
tfidf = dict(zip(vect_idf.get_feature_names_out(), vect_idf.idf_))

In [39]:
tfidf

{'00': 9.886132618174951,
 '0000000000000': 9.886132618174951,
 '005': 9.886132618174951,
 '01': 9.886132618174951,
 '01072017не': 9.886132618174951,
 '05': 9.480667510066787,
 '06082017': 9.886132618174951,
 '0пишет': 9.886132618174951,
 '0тличн': 9.886132618174951,
 '0хот': 9.886132618174951,
 '10': 6.62803608015347,
 '100': 7.247075288559693,
 '1000': 8.787520329506842,
 '10000': 9.886132618174951,
 '100817': 9.886132618174951,
 '100гб': 9.886132618174951,
 '100не': 9.886132618174951,
 '100шт': 9.886132618174951,
 '101': 9.886132618174951,
 '1010': 8.787520329506842,
 '1015': 9.886132618174951,
 '1015и': 9.886132618174951,
 '10й': 9.886132618174951,
 '10копеек': 9.886132618174951,
 '11062017': 9.886132618174951,
 '11111': 9.886132618174951,
 '11тр': 9.886132618174951,
 '11числ': 9.886132618174951,
 '12': 8.49983825705506,
 '1200': 9.886132618174951,
 '13': 9.886132618174951,
 '15': 8.49983825705506,
 '150': 9.192985437615006,
 '1500': 9.886132618174951,
 '15000тыс': 9.88613261817495

In [40]:
rt = vect_idf.vocabulary_.items()

In [41]:
vect_idf.idf_[vect_idf.vocabulary_['automating']]

9.886132618174951

In [42]:
len(tfidf)

9890

In [43]:
from collections import defaultdict

In [44]:
max_idf = max(vect_idf.idf_)

word2weight = defaultdict(
    lambda: max_idf,
    [(w, vect_idf.idf_[i]) for w, i in vect_idf.vocabulary_.items()])

In [45]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(100)  # Здесь 100 - размерность векторов в модели Word2Vec
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V.wv.key_to_index:
            vector_w2v += modelW2V.wv.get_vector(wrd) * 1
            n_w2v += 1
    if n_w2v != 0:
        return vector_w2v / n_w2v
    else:
        return vector_w2v


# Создать и обучить объект TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)  # Используем nltk для токенизации
tfidf_vectorizer.fit(df_train['Content'])

def get_vect_idf(txt, tfidf_vectorizer, modelW2V):
    vector_w2v = np.zeros(100)
    n_w2v = 0
    tokenized_text = word_tokenize(txt)  # Токенизация текста с помощью nltk
    for wrd in tokenized_text:
        if wrd in modelW2V.wv.key_to_index:  # Проверка наличия слова в модели Word2Vec
            iddf_ = tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[wrd]]
            vector_w2v += modelW2V.wv[wrd] * iddf_
            n_w2v += iddf_
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [46]:
from tqdm import tqdm_notebook

In [47]:
arr_vect = []
for txt in tqdm_notebook(df_train['Content']):
    arr_vect.append(get_vect_mean(txt))

arr_vect_valid = []
for txt in tqdm_notebook(df_val['Content']):
    arr_vect_valid.append(get_vect_mean(txt))
    
train_w2v = np.asarray(arr_vect)    
valid_w2v = np.asarray(arr_vect_valid)

  0%|          | 0/14461 [00:00<?, ?it/s]

  0%|          | 0/3099 [00:00<?, ?it/s]

In [48]:
lgr_w2v = LogisticRegression()

In [49]:
lgr_w2v.fit(train_w2v, df_train['Rating'].to_numpy())

LogisticRegression()

In [50]:
y_pred = lgr_w2v.predict(valid_w2v)

In [51]:
accuracy_score(df_val['Rating'].to_numpy(), y_pred)

0.7347531461761858

In [52]:
arr_vect = []
for txt in tqdm_notebook(df_train['Content']):
    arr_vect.append(get_vect_idf(txt, tfidf_vectorizer, modelW2V))

arr_vect_valid = []
for txt in tqdm_notebook(df_val['Content']):
    arr_vect_valid.append(get_vect_idf(txt, tfidf_vectorizer, modelW2V))
    
train_w2v = np.asarray(arr_vect)    
valid_w2v = np.asarray(arr_vect_valid)

  0%|          | 0/14461 [00:00<?, ?it/s]

  0%|          | 0/3099 [00:00<?, ?it/s]

In [53]:
lgr_w2v = LogisticRegression()
lgr_w2v.fit(train_w2v, df_train['Rating'].to_numpy())
y_pred = lgr_w2v.predict(valid_w2v)

In [54]:
accuracy_score(df_val['Rating'].to_numpy(), y_pred)

0.7312036140690545

#### get_vect_mean: 
В этом подходе модель modelW2V (Word2Vec) для получения векторных представлений слов из текста. Мы инициализируем вектор нулями и добавляеv векторы слов, которые присутствуют в модели modelW2V. Затем мы делим их на общее количество слов в тексте n_w2v для получения среднего вектора текста.
#### get_vect_idf: 
В этом подходе мы используем модель modelW2V и объект tfidf_vectorizer (TfidfVectorizer) для получения взвешенного среднего вектора текста. Здесь мы используете оценки важности слова в документе (IDF).

- Причины различия в результатах могут быть следующими:

1. Различные представления текста: В первом подходе мы используем среднее значение векторов слов, что может привести к потере информации о последовательности слов в тексте. Во втором подходе мы учитываем важность слов с помощью IDF, что может помочь учесть контекст и семантику текста.
2. Размерность векторов: Возможно, размерность векторов в модели modelW2V и объекте tfidf_vectorizer различается, что может повлиять на качество представления текста.

Но для данной конкретной задачи использование IDF для взвешивания векторов Word2Vec не дало существенного прироста в качестве.